In [1]:
import json
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Сформировать множество из разных файлов

In [2]:
# df = pd.read_csv("../../Data/Raw/one_cmd_v3_03062022.csv", sep=";")
df = pd.read_csv("../../Data/Raw/one_cmd_v4_05072022.csv", sep=";")
print(df.shape)
print(len(df[df['x'].isna()]), "nan texts")
df = df[~df['x'].isna()]
df.drop("delayed", axis=1, inplace=True)
df.drop_duplicates(inplace=True)
df.shape

(55871, 17)
0 nan texts


(55871, 16)

# предобработка

In [3]:
# в случае изменения функции, изменить её и в обработке длинных команд в Analyze_complex_commands.ipynb
#punct_characters = re.escape(".,!?:;-#$*()[]'\"/|^%~`_+-")
import re
def removePunctETC(value):
    if type(value) is float:
        return value
    value = value.lower()
    value = re.sub("[^\w\s°]", " ", value)
    value = re.sub(" {2,}", " ", value)
    return value
df["x"] = df["x"].map(removePunctETC)
df.drop_duplicates(inplace=True)  # не делаю, чтоб сплит совпал с обычными
df.shape

(55828, 16)

# сплит

In [4]:
# сплит сгенерированной части на трейн и валид 
print(df["type"].unique())
traindf = df[~df["type"].isin(['students', 'crowdsource'])]

validIndex = set()
for c in traindf.columns:
    if c in ["x", "y_name", "type"]:
        continue
    train_x, valid_x = train_test_split(traindf.index, test_size=0.03, random_state=69, shuffle=True, stratify=traindf[c])
    validIndex |= set(valid_x)
print("valid percent", len(validIndex) / df.shape[0])
subsets = ["valid" if i in validIndex else "train" for i in range(df.shape[0])]
df["subset"] = subsets
df.loc[df["type"].isin(['students', 'crowdsource']),"subset"] = "test"

['generator']
valid percent 0.23837500895607938


In [5]:
for c in df.columns:
    if c in ["x", "y_name", "subset", "type"]:
        continue
    trains_counts = df.loc[df["subset"]=="train", c].value_counts().to_dict()
    valid_counts = df.loc[df["subset"]=="valid", c].value_counts().to_dict()
    test_counts = df.loc[df["subset"]=="test", c].value_counts().to_dict()
    print(c)
    for k in trains_counts:
        if k in test_counts:
            print(k, trains_counts[k], valid_counts[k], test_counts[k], np.round(valid_counts[k] / trains_counts[k], 3), np.round(test_counts[k] / trains_counts[k],2))
        elif k in valid_counts:
            print(k, trains_counts[k], valid_counts[k], np.round(valid_counts[k] / trains_counts[k]), 3)
        else:
            print(k, trains_counts[k])
    print()

y
9 5086 684 0.0 3
2 4655 1331 0.0 3
7 4312 1688 0.0 3
11 4222 1103 0.0 3
3 4071 885 0.0 3
8 4028 1972 0.0 3
6 3902 1284 0.0 3
10 3808 1562 0.0 3
4 3799 989 0.0 3
5 2700 1005 0.0 3
1 2177 457 0.0 3
0 36 11 0.0 3

action
5 7870 1874 0.0 3
6 7363 2451 0.0 3
0 5098 687 0.0 3
4 4655 1331 0.0 3
9 4189 1540 0.0 3
10 4146 1623 0.0 3
8 4106 1620 0.0 3
7 3168 1380 0.0 3
2 1921 403 0.0 3
3 256 54 0.0 3
1 16 7 0.0 3
11 5 1 0.0 3
12 3

direction
0 28094 9309 0.0 3
3 3524 851 0.0 3
1 2185 602 0.0 3
4 1742 407 0.0 3
7 1677 410 0.0 3
5 1668 420 0.0 3
6 1646 414 0.0 3
8 1593 402 0.0 3
2 667 156 0.0 3

meters
0 33055 10956 0.0 3
6 850 165 0.0 3
1 847 163 0.0 3
10 833 170 0.0 3
3 825 155 0.0 3
5 817 163 0.0 3
4 813 180 0.0 3
11 807 185 0.0 3
2 807 175 0.0 3
8 795 156 0.0 3
9 791 162 0.0 3
12 789 159 0.0 3
7 767 182 0.0 3

degshours
0 34926 11097 0.0 3
7 363 71 0.0 3
12 354 94 0.0 3
6 350 73 0.0 3
3 348 78 0.0 3
11 341 68 0.0 3
10 341 72 0.0 3
2 338 79 0.0 3
5 337 88 0.0 3
18 333 79 0.0 3
4 331 86 0.0 3


In [5]:
df.to_csv("../../Data/Interim/one_cmd_v4_05072022_low_split.csv", index=False)
#df = pd.read_csv("../../Data/Interim/merged_with_labels_11_patterns_split.csv")
#df.sample(2000).to_csv("../../Data/Interim/short_split.csv", index=False)

# folds

In [6]:
from sklearn.model_selection import StratifiedKFold
df = pd.read_csv("../../Data/Interim/one_cmd_v4_05072022_low_split.csv")
df_old_folds = pd.read_csv("../../Data/Interim/merged_with_labels_11_patterns_split.csv")
df["fold"]=-1
df[df["subset"]=="test"]["y"].value_counts()

Series([], Name: y, dtype: int64)

In [7]:
df[df["subset"]=="test"]["action"].value_counts()

Series([], Name: action, dtype: int64)

In [8]:
df = pd.concat([df, df_old_folds[df_old_folds["subset"]=="test"]])

In [9]:
(df.isna()).any()

x            False
y_name       False
y            False
action       False
direction    False
meters       False
degshours    False
object1      False
nearest      False
relation1    False
object2      False
relation2    False
object3      False
self         False
gaze         False
type         False
subset       False
fold         False
dtype: bool

In [10]:
df["x"] = df["x"].map(removePunctETC)

In [11]:
df.to_csv("../../Data/Interim/one_cmd_v4_05072022_low_split.csv", index=False)